# Ansible and Windows

2015-12-07
Carsten Langer

# Agenda
* How it should work
* Issues with PowerShell versions 2 and 3
* Issues with SSL certificate handling
* Issues with Windows Server 2012
* Solutions to the SSL certificate handling

# Concept

* Windows controlled client
* Ansible runs on GNU/Linux [[1]](http://docs.ansible.com/ansible/intro_windows.html#reminder-you-must-have-a-linux-control-machine)
* "Manage Windows like Windows",  
    same as "manage Linux like Linux" [[2]](http://www.ansible.com/windows):
    * PowerShell instead of Python scripts
    * Native PowerShell remoting
        * Windows Remote Management (winrm) service
        * HTTP(S), instead of SSH

# How it should work

* Ansible -> HTTPS -> winrm
* Send user/pass via Basic Authentication = clear text
    * Domain machines can use Kerberos
* Send PS script
* Execute PS script

# Requirements

On controlled Windows machine:

* Installing and enabling the Windows Remote Management service.
* Enabling the PowerShell remoting.
* Enabling an HTTPS listener
    * provide with SSL certificate and corresponding private key.
* Allowing Basic Authentication.
* Opening the Firewall (5986 for HTTPS, 5985 for HTTP).

On Ansible control machine:

* pywinrm >= 0.1.1.
* Accept the SSL certificate from Windows machine.

# Doesn't work out of the box

There are:
* Issues with PowerShell versions 2 and 3
* Issues with SSL certificate handling
* Issues with Windows Server 2012

# Issues with PowerShell versions 2 and 3

## Problem

* most Ansible win modules require PS3+
* PS3 requires .Net4
* provided script [upgrade_to_ps3.ps1](https://github.com/cchurch/ansible/blob/devel/examples/scripts/upgrade_to_ps3.ps1) has check for .Net4 commented out and check is broken.
* Having PS3 is not enough:
    * After upgrade to PS3 you run Windows Management Framework 3
    * You need another hotfix.
    
## My solution

* Manually install .Net4.5
* Manually install Windows Management Framework 4.0, which includes PowerShell version 4.

# Issues with SSL certificate handling

## Problem

* Provided script [ConfigureRemotingForAnsible.ps1](https://github.com/ansible/ansible/blob/devel/examples/scripts/ConfigureRemotingForAnsible.ps1) generates
    * new private key
    * new self-signed SSL certificate
    * new Windows Remote Management HTTPS listener to use the certificate
* Ansible uses pywinrm to connect from Python to the winrm service via HTTPS.
* pywinrm uses Python's ssl module to handle the SSL certificate
* Since v2.7.9 Python checks the SSL certificate
* It was a bug in earlier versions of Python not to check the certificate [[3]](https://www.python.org/dev/peps/pep-0476/).

--> Since the SSL certificate is self-signed, the verification fails, which is the correct behavior for untrusted certificates. Thus Ansible will report a SSL certificate verification error.

```
fatal: [my.windows.machine.net]: FAILED! => {"failed": true, "msg": "ERROR! ssl: 500 WinRMTransport. [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:590)"}
```

# Options to get SSL working

From best to worse:

* Deploy an official SSL certificate to the Windows machine.
* Use the self-signed certificate on the Windows machine, and make the Ansible/pywinrm/ssl/Python software stack accept the certificate by adding it to the store of trusted certificated.
* Use the self-signed certificate on the Windows machine, but switch off certificate validation.
    * pro: still transport channel encryption
    * con: no authentication of remote side
* Use unencrypted HTTP instead of HTTPS.
    * pro: fallback if you are desperate
    * con: no transport channel encryption, thus user/pass in cleartext
    * con: no authentication of remote side

Details per solution later in slideset.

# Issues with Windows Server 2012

## Problems

Windows Server 2012 seems to behave differently:
* The scripts only runs as Administrator, not as a user with admin rights.
* Running the _ConfigureRemotingForAnsible.ps1_ script does not lead to a working configuration
* The script does not fully allow remote access. Verify with:

```
PS C:\ winrm get winrm/config/Service
    Service
...
        AllowRemoteAccess = false
```

* The winrm service does not deliver the full certificate chain of the own certificate and the intermediate certificate to the Ansible client.
* In earlier versions of Windows both certificates are delivered.
* If intermediate certificate delivery would be required, Ansible again runs into the certificate verification error.

## Windows Server 2012 Solutions

Full solution pending. Some ideas are:
* Run the scripts as Administrator.
* Manually allow remote access, e.g. via `Enable-PSRemoting`.
* On the missing delivery of the intermediate certificate I have no idea yet how to fix this.
* As workaround, you can use the 3rd best option described below.

Your ideas?

# Best solution: official certificate

* Create an official certificate
* Join together the private key, the server certificate and the intermediate certificate to a PKCS#12 bundle.

```
# openssl pkcs12 -export -out windows_key_and_certificate.pfx -inkey server_rsa_private_key.key -in cert.cer.txt -certfile Symantec_Class_3_Secure_Server_CA_-_G4.cer

Enter pass phrase for server_rsa_private_key.key:

Enter Export Password:

Verifying - Enter Export Password:

#
```

# Best solution: official certificate

* On Windows, import the bundle using MMC [[4]](https://technet.microsoft.com/en-us/library/cc780916%28v=ws.10%29.aspx).
* Import the bundle to the __Local Computer Personal Certificate__ store.
* The store shall show your official certificate with a key symbol, and the intermediate certificate without a key symbol.
* Double check from Windows PowerShell that the import worked:

```
PS C:\> Get-ChildItem cert:\LocalMachine\My
    Directory: Microsoft.PowerShell.Security\Certificate::LocalMachine\My
Thumbprint                                Subject                                                                      
----------                                -------
FF67367C5CD4DE4AE18BCCE1D70FDABD7C866135  CN=Symantec Class 3 Secure Server CA - G4, OU=Symantec Trust Network, O=Symantec...
B8E77FFF760AE933C141510ABC2ACD8BF8B1D5C4  CN=my.windows.machine.net, OU=For Intranet Use ...
```

# Best solution: official certificate

* On Windows run a tweaked PS script [ConfigureRemotingForAnsibleUsingExistingCertificate.ps1](https://github.com/carsten-langer/ansible-tweaks/blob/master/ConfigureRemotingForAnsibleUsingExistingCertificate.ps1), based on Ansible's original script _ConfigureRemotingForAnsible.ps1_, but is modified to take the official SSL and private key in use instead of creating a new self-signed certificate.
* You may have to run `Set-ExecutionPolicy -ExecutionPolicy Unrestricted -Scope CurrentUser` first to allow running of unsigned scripts.

After that, Ansible can connect to the Windows machine.

# 2nd best solution: Make Ansible accept the self-signed certificate

Idea:

* export the self-signed certificate from the Windows machine
* tell Ansible explicitly to trust this certificate

Unfortunately, it seems not clear how to tell Ansible to trust a certain certificate, or where the right place would be anyway, in Ansible, in pywinrm, in a Python SSL library or in the system GNU/Linux certificate store. A lengthy discussion without a solution is at [[5]](https://github.com/ansible/ansible/issues/10294).

# 2nd best solution: Make Ansible accept the self-signed certificate

So far I managed to store the certificate such that Python accesses it (On Ubuntu `cp my-self-signed-cert.crt /usr/local/share/ca-certificates/ ; dpkg-reconfigure ca-certificates`). I verify that it is accessed by:
```
# strace -f python /<path>/ansible-playbook -i windows windows.yml 2>&1 | grep "/usr/lib/ssl/certs"
[pid 25000] stat("/usr/lib/ssl/certs/ac9f2638.0",  <unfinished ...>
[pid 25000] open("/usr/lib/ssl/certs/ac9f2638.0", O_RDONLY) = 15
[pid 25000] stat("/usr/lib/ssl/certs/ac9f2638.1", 0x7ffe2954e290) = -1 ENOENT (No such file or directory)

# ll /usr/lib/ssl/certs/ac9f2638.0
lrwxrwxrwx 1 root root 61 Dec  7 13:01 /usr/lib/ssl/certs/ac9f2638.0 -> my-self-signed-cert.pem
``` 

However, I still receive the the SSL verification error.

Your ideas?

# 3rd best solution: Use self-signed certificate, but let Ansible ignore it

* Still use HTTPS transport encryption, so username/password and other info is not visible on the network.
* But SSL certificate is not verified, thus no authentication of remote machine (inventory config error, man-in-the-middle).

Solution since November 2015:
* Discussion at [[6]](https://github.com/ansible/ansible/pull/12687).
* Combination of pywinrm version 0.1.1 and newest Ansible 2 development release.
* Enables a new variable `ansible_winrm_server_cert_validation`.
* Set to `ignore`, pywinrm will not verify the SSL certificate.

So your group_vars/windows.yml file could look like this:
```
ansible_user: cloud
ansible_password: Temp1234
ansible_connection: winrm
ansible_port: 5986 # 5986 = HTTPS, 5985 = HTTP
ansible_winrm_server_cert_validation: ignore # requires pywinrm>=0.1.1 and recent Ansible 2 development version
```

With this setting, Ansible will use HTTPS, but not validate the SSL certificate, thus Windows can use a self-signed certificate.

# Worst solution: Do not use transport encryption

* Worst from security perspective.
* Plain HTTP on port 5985 instead of HTTPS on port 5986.
* No transport encryption is used --> username, password and all info in clear text

Fallback use cases:
* If you do not have the official certificate which you need for the best solution
* If you do not have Ansible 2 which you need for the 3rd best solution.

Following the discussion [[7]](https://social.msdn.microsoft.com/Forums/windowsserver/en-US/fb154aea-33ee-4182-a345-66f88a6769bc/allowunecrypted-winrm-property?forum=windowssecurity), on Windows PS use
```
winrm set winrm/config/service '@{AllowUnencrypted="true"}'
```
The `' '` are important, as otherwise for Windows Server 2012 the command will fail.

Then set the variable `ansible_ssh_port` to `5985`. Do not use `ansible_port` as it is ignored in Ansible 1.9. Your _group_vars/windows.yml_ file could look like this:
```
ansible_user: cloud
ansible_password: Temp1234
ansible_connection: winrm
ansible_ssh_port: 5985 # 5986 = HTTPS, 5985 = HTTP
```